In [4]:
# Because of out of the box error - Remove any previous Spark folder to avoid conflicts
!rm -rf spark-3.5.3-bin-hadoop3

# Try using an alternate mirror URL (the Apache archive mirror) to be sure:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

# Extract the tarball
!tar xf spark-3.5.3-bin-hadoop3.tgz


In [5]:
# Check installation due to previous errors
!ls /content/spark-3.5.3-bin-hadoop3/python/lib/


py4j-0.10.9.7-src.zip  PY4J_LICENSE.txt  pyspark.zip


In [6]:
import os

# Define your Spark version
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION'] = spark_version

# Install Java and Spark
!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Use an alternate mirror URL from the Apache archive
!wget -q https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Verify that SPARK_HOME looks correct:
print("SPARK_HOME:", os.environ["SPARK_HOME"])
!ls -l ${SPARK_HOME}

# Check if py4j is present
!find ${SPARK_HOME} -name "py4j*.zip"

# (Optional) Set PYSPARK_SUBMIT_ARGS if needed for extra packages
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3 pyspark-shell'

# Now, initialize findspark explicitly with the SPARK_HOME
import findspark
findspark.init(os.environ["SPARK_HOME"])


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
SPARK_HOME: /content/spark-3.5.3-bin-hadoop3
total 152
drwxr-xr-x 2 1000 1000  4096 Sep  9  2024 bin
drwxr-xr-x 2 1000 1000  4096 Sep  9  2024 conf
drwxr-xr-x 6 1000 1000  4096 Sep  9  2024 data
drwxr-xr-x 4 1000 1000  4096 Sep  9  2024 examples
drwxr-xr-x 2 1000 1000 16384 Sep  9  2024 jars
drwxr-xr-x 4 1000 1000  4096 Sep  9  2024 kubernetes
-rw-r--r-- 1 1000 1000 22916 Sep  9  2024 LICENSE
drwxr-xr-x 2 1000 1000  4096 Sep  9  2024 licenses
-rw-r--r-- 1 1000 1000 57842 Sep  9  2024 NOTICE
drwxr-xr-x 9 1000 1000  4096 Sep  9  2024 python
drwxr-xr-x 3 1000 1000  4096 Sep  9  2024 R
-rw-r--r-- 1 1000 1000  4605 Sep  9  2024 README.md
-rw-r--r-- 1 1000 1000   166 Sep  9  2024 RELEASE
drwxr-xr-x 2 1000 1000  4096 Sep  9  2024 sbin
drwxr-xr-x 2 1000 1000  4096 Sep  9  2024 yarn
/content/spark-3.5.3-bin-h

In [7]:
# Double check installation before proceeding
!ls /content/spark-3.5.3-bin-hadoop3/python/lib/

py4j-0.10.9.7-src.zip  PY4J_LICENSE.txt  pyspark.zip


In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [10]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [13]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Assuming the column containing the year information is named 'date'
# and you want to extract the year from it
from pyspark.sql.functions import year, round  # Import the round function from pyspark.sql.functions

avg_price_per_year = home_sales_df.filter(home_sales_df.bedrooms == 4) \
    .groupBy(year(home_sales_df.date).alias("year")) \
    .agg({"price": "avg"}) \
    .withColumnRenamed("avg(price)", "average_price")

avg_price_per_year = avg_price_per_year.withColumn("average_price", round(avg_price_per_year["average_price"], 2)) # Use the round function from pyspark.sql.functions

In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_per_year_built = home_sales_df.filter((home_sales_df.bedrooms == 3) & (home_sales_df.bathrooms == 3)).groupBy("date_built").agg({"price": "avg"}).withColumnRenamed("avg(price)", "average_price")
avg_price_per_year_built = avg_price_per_year_built.withColumn("average_price", round(avg_price_per_year_built["average_price"], 2))

In [17]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_per_year_built_2floors = home_sales_df.filter((home_sales_df.bedrooms == 3) & (home_sales_df.bathrooms == 3) & (home_sales_df.floors == 2) & (home_sales_df.sqft_living >= 2000)).groupBy("date_built").agg({"price": "avg"}).withColumnRenamed("avg(price)", "average_price") # Changed 'year_built' to 'date_built' in groupBy clause
avg_price_per_year_built_2floors = avg_price_per_year_built_2floors.withColumn("average_price", round(avg_price_per_year_built_2floors["average_price"], 2))

In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_price_per_view = home_sales_df.groupBy("view").agg({"price": "avg"}).withColumnRenamed("avg(price)", "average_price")
avg_price_per_view = avg_price_per_view.withColumn("average_price", round(avg_price_per_view["average_price"], 2))


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.05673527717590332 seconds ---


In [19]:
# 7. Cache the the temporary table home_sales.
home_sales_df.createOrReplaceTempView("home_sales")

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

False

In [21]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_price_per_view_cached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM home_sales GROUP BY view HAVING average_price >= 350000 ORDER BY view DESC")
avg_price_per_view_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 2.683929920196533 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# and write the DataFrame out as a parquet file.
home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet")


In [23]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [25]:
# 12. Create a temporary table for the parquet data.
# Assigning the parquet DataFrame to the variable home_sales_partitioned_df
home_sales_partitioned_df = parquet_df
home_sales_partitioned_df.createOrReplaceTempView("home_sales_partitioned")

In [26]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
avg_price_per_view_parquet = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM home_sales_partitioned GROUP BY view HAVING average_price >= 350000 ORDER BY view DESC")
avg_price_per_view_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9124617576599121 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

In [29]:
# 16. Stop the SparkSession
spark.stop()